In [ ]:
import numpy as np
import pandas as pd
import os 
import tensorflow as tf
import tensorflow.keras as keras
from keras import Model
from keras.applications.densenet import DenseNet121
from keras.applications import vgg16
from keras.preprocessing import image
from keras.applications.densenet import preprocess_input, decode_predictions
from keras.layers import GlobalMaxPooling2D
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pathlib
from sklearn.metrics.pairwise import linear_kernel
import tensorflow_hub as hub
from PIL import Image

In [ ]:
path = 'archive/'
dataset_path = pathlib.Path(path)
dirs_names = os.listdir(dataset_path) # list content of dataset
dirs_names

styles_df = pd.read_csv(path + "styles.csv", nrows=20000, error_bad_lines=False) # Read n product and drop bad lines 
styles_df['image'] = styles_df.apply(lambda x: str(x['id']) + ".jpg", axis=1) # Make image column contains (id.jpg)
styles_df.head()

C:\Users\samis\AppData\Local\Temp\ipykernel_26868\1805511326.py:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  styles_df = pd.read_csv(path + "styles.csv", nrows=20000, error_bad_lines=False) # Read n product and drop bad lines
Skipping line 6044: expected 10 fields, saw 11
Skipping line 6569: expected 10 fields, saw 11
Skipping line 7399: expected 10 fields, saw 11
Skipping line 7939: expected 10 fields, saw 11
Skipping line 9026: expected 10 fields, saw 11
Skipping line 10264: expected 10 fields, saw 11
Skipping line 10427: expected 10 fields, saw 11
Skipping line 10905: expected 10 fields, saw 11
Skipping line 11373: expected 10 fields, saw 11
Skipping line 11945: expected 10 fields, saw 11
Skipping line 14112: expected 10 fields, saw 11
Skipping line 14532: expected 10 fields, saw 11
Skipping line 15076: expected 10 fields, saw 12



,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg


In [ ]:
top_25 = styles_df['articleType'].value_counts().sort_values(ascending=False).head(25)
top_25_list = top_25.index
print(top_25_list)


# get only data with the more used 
styles_df_processed = styles_df.loc[styles_df['articleType'].isin(top_25_list), :]




Index(['Tshirts', 'Shirts', 'Casual Shoes', 'Watches', 'Sports Shoes',
       'Kurtas', 'Tops', 'Handbags', 'Heels', 'Sunglasses', 'Flip Flops',
       'Wallets', 'Sandals', 'Briefs', 'Belts', 'Backpacks', 'Socks',
       'Perfume and Body Mist', 'Formal Shoes', 'Jeans', 'Trousers', 'Shorts',
       'Flats', 'Bra', 'Sarees'],
      dtype='object')


In [ ]:
#function for Preprocessing the data 

# Inputs must respect these conditions : 

   # be four dimensional Tensors of the shape (batch_size, height, width, num_channels). Note that the model expects images with channels_last property. num_channels must be 3.
   # be resized to 224x224 resolution.
   # have pixel values in the range [-1, 1].

def img_path(img):
    """ Take image name(id) and return the complete path of it """
    return path + 'images/' + img

def preprocess_image(image_path):
    # Load the image from file
    image = tf.io.read_file(image_path)
    
    # Decode the image and convert it to a tensor
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # Convert the data to a four-dimensional tensor
    image = tf.expand_dims(image, axis=0)
    
    # Resize the data to a resolution of 224x224
    image = tf.image.resize(image, (224, 224))
    
    # Normalize the pixel values to the range [-1, 1]
    image = (image / 127.5) - 1
    
    # Add a channels dimension to the data
    image = tf.transpose(image, (0, 3, 1, 2))
    
    return image


In [ ]:
def load_labels_from_files(label_files):
    path = 'archive/'
    dataset_path = pathlib.Path(path)
    dirs_names = os.listdir(dataset_path) # list content of dataset
    # Create an empty list to store the labels
    labels = []

    # Iterate over the label files
    styles_df = pd.read_csv(path + "styles.csv", nrows=10000, error_bad_lines=False) # Read n product and drop bad lines 
    styles_df['image'] = styles_df.apply(lambda x: str(x['id']) + ".jpg", axis=1) # Make image column contains (id.jpg)

    return labels

def load_images_from_files(image_files):
  
    # Create an empty list to store the images
    images = []
    error_num =[]
    # Iterate over the image files
    for file in image_files:
        # Load the image using PIL
        try:
            
            image = preprocess_image(img_path(file))
            images.append(image)
        except FileNotFoundError:
            print("Could not find the specified file : ",path + 'images/')
            error_num.append(file)
            continue
        except ValueError:
            print("not enough row and columns")
            error_num.append(file)
            continue
        except Exception as e:
            error_num.append(file)
            print("An error occurred:", e)
        
        

    # Convert the list of images to a numpy array
    images = np.array(images)

    return images,error_num

images,error_num = load_images_from_files(styles_df_processed['image'].tolist())

def get_labels(dataset,error_list):
    res = []
    for rows in dataset:
        if rows['image'] not in error_list:
            res.append(rows['articleType'])
        else: 
            pass
    return res
 
labels = get_labels(styles_df_processed,error_num)
print(len(images))
print(len(labels))


not enough row and columns
not enough row and columns


MemoryError: Unable to allocate 8.97 GiB for an array with shape (15989, 1, 3, 224, 224) and data type float32

In [ ]:
print(len(images))
print(len(labels))

15989
15991


In [ ]:
from sklearn.model_selection import train_test_split

IMAGE_SHAPE = (224, 224)
# Load the pre-trained model from TensorFlow Hub
classifier_model = "https://tfhub.dev/sayakpaul/vit_b8_classification/1"
model = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])



print(model.summary)


<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x0000020817CE9F00>>


In [ ]:



# Replace the top layers of the model
num_classes = 10  # Number of classes in your dataset
x = model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
predictions = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
model = tf.keras.Model(inputs=model.input, outputs=predictions)

# Freeze the weights of the pre-trained layers
for layer in model.layers[:-3]:
    layer.trainable = False

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

# Train the model on your dataset
model.fit(x_train, y_train, epochs=10, batch_size=32)

ValueError: Found input variables with inconsistent numbers of samples: [15989, 15991]

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(test_loss," ",test_acc)